In [2]:
import numpy as np
import pandas as pd
import re
import editdistance
from sklearn.metrics import accuracy_score

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
all_sents = pd.read_csv("data\MoodyLyrics4Q_featurized.txt", index_col=0)
sentiment = all_sents.iloc[:, 3:]

In [4]:
vecs = []
for i in range(len(sentiment)):
    # print(re.sub(r"\[|\]|\n|\'", "", sentiment[["LYRICS_VECTOR"]].iloc[i][0]))
    long_string = re.sub(r"\[|\]|\n|\t|\'", "", sentiment[["LYRICS_VECTOR"]].iloc[i][0]).replace("  ", " ").split(" ")
    e = []
    for item in long_string:
        if len(item) != 0:
            e += [float(item)]
    vecs += [e]


In [5]:
lyric_vecs = np.array(vecs)

In [6]:
title_vecs = np.array(vecs)

In [7]:
all_vecs = np.hstack((title_vecs, lyric_vecs))

In [8]:
big_boi = np.hstack((sentiment.iloc[:,[7, 9, 10, 12, 13, 14, 15, 28, 31, 32]].to_numpy(), all_vecs))

In [65]:
big_boi.shape

(1935, 610)

In [10]:
y = pd.factorize(sentiment.EMOTION)[0]

In [66]:
sentiment.shape


(1935, 34)

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

models = [
           LogisticRegression(solver = 'lbfgs'),
           ExtraTreesClassifier(),
           NuSVC()
        ]
for m in models:
    X_train, X_test, y_train, y_test = train_test_split(big_boi, y, train_size=0.9)
    m.fit(X_train, y_train)
    pred = m.predict(X_test)
    print(accuracy_score(y_test, pred))

0.6134020618556701
0.5309278350515464
0.5824742268041238


In [12]:
pred = lg.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
print(accuracy_score(y_test, pred))

0.6030927835051546


In [15]:
artists = [s.lower() for s in all_sents.iloc[:,1]]

In [16]:
our_artists = pd.read_csv("data/raw/SpotifyData.csv")[["artist"]].to_numpy()

In [17]:
idx = []
idx_2 = []
i = 0
for j, our_artist in enumerate(our_artists):
    # print(our_artist)
    skip = 0
    while i < len(artists) and editdistance.eval(str(our_artist[0]), artists[i]) > 4:
        i = i + 1
        if i >= len(artists):
            i = j + 1
            skip = 1
            break
        # print(str(our_artist[0]), artists[i])
    if not skip:
        idx += [i]
        idx_2 += [j]
len(idx)

1764

In [18]:
np.hstack((np.array(artists)[idx].reshape(len(idx), 1), np.array(our_artists[:,0][idx_2].reshape(len(idx_2), 1))))

array([['george michael', 'george michael'],
       ['rob zombie', 'rob zombie'],
       ['katatonia', 'katatonia'],
       ...,
       ['deine lakaien', 'deine lakaien'],
       ['fatboy slim', 'fatboy slim'],
       ['cooler kids', 'cooler kids']], dtype=object)

In [26]:
big_boi[idx].shape

(1764, 610)

In [28]:
audio = pd.read_csv("data/preprocessed/spotify-data-preprocessed.csv").to_numpy()[idx_2]

In [29]:
multimodality_is_here = np.hstack((audio[:,1:-4], big_boi[idx]))

In [30]:
multimodality_is_here.shape

(1764, 623)

In [34]:
from sklearn.svm import NuSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

X_train, X_test, y_train, y_test = train_test_split(multimodality_is_here, y[idx], train_size=0.9)
lg = ExtraTreesClassifier( n_estimators=1000)
lg.fit(X_train, y_train)
pred = lg.predict(X_test)

print(accuracy_score(y_test, pred))

0.6666666666666666


In [72]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
accs = []
confs = []

# Training / Testing
for train_index, test_index in skf.split(multimodality_is_here, y[idx]):
    X_train, X_test = multimodality_is_here[train_index], multimodality_is_here[test_index]
    y_train, y_test = y[idx][train_index ], y[idx][test_index]

    model = ExtraTreesClassifier(n_estimators=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accs += [accuracy_score(y_test, y_pred)]
    # confs += [confusion_matrix(y_test, y_pred)]

In [73]:
np.mean(accs)

0.6621468926553671

In [74]:
fusion_audio = audio[:,1:-4]
fusion_text = big_boi[idx]

accs= []

for train_index, test_index in skf.split(multimodality_is_here, y[idx]):
    audio_train, audio_test = fusion_audio[train_index], fusion_audio[test_index]
    text_train, text_test = fusion_text[train_index], fusion_text[test_index]
    y_train, y_test = y[idx][train_index ], y[idx][test_index]

    model1 = LogisticRegression()
    model2 = LogisticRegression()
    model1.fit(audio_train, y_train)
    model2.fit(text_train, y_train)

    model3 = LogisticRegression()
    audio_predited = np.array(model1.predict_proba(audio_train))
    text_predicted = np.array(model2.predict_proba(text_train))

    fking = np.hstack((audio_predited, text_predicted))
    model3.fit(fking, y_train)

    abc1 = np.array(model1.predict_proba(audio_test))
    abc2 = np.array(model2.predict_proba(text_test))
    fking2 = np.hstack((abc1, abc2))
    abc3 = model3.predict(fking2)

    accs += [accuracy_score(y_test, abc3)]
print(np.mean(accs))

0.703508602978942


In [76]:
accs

[0.7570621468926554,
 0.6949152542372882,
 0.655367231638418,
 0.7175141242937854,
 0.6988636363636364,
 0.7102272727272727,
 0.6477272727272727,
 0.6704545454545454,
 0.7215909090909091,
 0.7613636363636364]